# Baseline model

This notebook demonstrates the baseline model for estimating the respiratory rate from a video of a person's chest. The model uses the Viola Jones algorithm to detect the face in the first frame of the video. It then calculates the region of interest (ROI) based on the face position and size. The model calculates the average pixel intensity in the chest region and applies a fast Fourier transform (FFT) to the signal. The respiratory rate is estimated by identifying the frequency that exhibits the highest amplitude within the FFT spectrum.

In [ ]:
import os
import cv2
import respiratory_extraction.utils as utils

In [ ]:
data_path = os.path.join(os.getcwd(), '..', 'data', 'subjects')
dataset = utils.Dataset(data_path)

subject = 'Proband16'
scenario = '101_natural_lighting'

frames, video_params = dataset.read_video_gray(subject, scenario)

## Detect the face in the first frame and calculate the region of interest

In [ ]:
import matplotlib.pyplot as plt
import respiratory_extraction.utils as utils

faces = utils.detect_faces(frames[0])
if len(faces) == 0:
    raise ValueError('No face detected in the first frame')
elif len(faces) > 1:
    raise ValueError('Multiple faces detected in the first frame')

# First face position and size
face1 = faces[0]

# Calculate the region of interest (ROI) based on the face
roi = utils.roi_from_face(face1[0], face1[1], face1[2], face1[3])

In [ ]:
# Detect the face in the first frame with Viola Jones algorithm
frame1 = frames[0].copy()

# Draw a rectangle around the face
cv2.rectangle(frame1, (face1[0], face1[1]), (face1[0] + face1[2], face1[1] + face1[3]), (255, 0, 0), 2)

# Draw a rectangle around the chest region
cv2.rectangle(frame1, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (255, 0, 0), 2)

plt.figure(figsize=(5, 5))
plt.imshow(frame1, cmap='gray')
plt.title('Face and chest detection')
plt.show()

## Calculate the average pixel intensity in the chest region

In [ ]:
import respiratory_extraction.models.baseline as baseline

# Calculate the average pixel intensity in the chest region
average_pixels = baseline.average_pixel_intensity(frames, roi)

In [ ]:
import plotly.express as px

# Plot the average pixel intensity in the chest region
fig = px.line(x=range(len(average_pixels)), y=average_pixels,
              title='Average pixel intensity in the chest region')
fig.show()

## Calculate the respiratory rate

In [ ]:
# Calculate the fast fourier transform of the average pixel intensity
pixels_fft, freq = baseline.calculate_fft(average_pixels, video_params.fps, min_freq=0.1, max_freq=0.6)

In [ ]:
import numpy as np

# Plot the filtered frequency spectrum
fig = px.line(x=freq, y=np.abs(pixels_fft), title='Filtered Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
max_freq, breathing_rate = baseline.calculate_respiratory_rate(pixels_fft, freq)
max_freq, breathing_rate

## Evaluate the breathing rate estimation

In [ ]:
# Read the ground truth data from unisens directory
gt_signal, gt_sample_rate = dataset.read_unisens_entry(subject, scenario, '3_Thorax')

In [ ]:
# Plot the data with the peaks
fig = px.line(x=range(len(gt_signal)), y=gt_signal, title='Ground truth thorax signal')
fig.show()

In [ ]:
# Calculate the fast fourier transform of the ground truth signal
gt_fft, gt_freq = baseline.calculate_fft(gt_signal, gt_sample_rate)
gt_max_freq, gt_breathing_rate = baseline.calculate_respiratory_rate(gt_fft, gt_freq)
gt_max_freq, gt_breathing_rate

In [ ]:
# Calculate the MAE between the ground truth and the estimated breathing rate
absolute_error = abs(gt_breathing_rate - breathing_rate)
absolute_error

## Calculate the breathing rate with the Autocorrelation Advanced Method

In [ ]:
acf_adv_freq = baseline.acf_adv(average_pixels, video_params.fps, min_freq=0.1, max_freq=0.6)
acf_adv_breathing_rate = acf_adv_freq * 60

acf_adv_freq, acf_adv_breathing_rate